### Model Fitting Notebook

In [1]:
#importing libraries

import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
import imblearn
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

np.random.seed(1)

In [2]:
X_train = pd.read_csv('phoneme_Xtrain.csv')
y_train = pd.read_csv('phoneme_ytrain.csv')
X_test = pd.read_csv('phoneme_Xtest.csv')
y_test = pd.read_csv('phoneme_ytest.csv')

In [3]:
X_train.head(10)

,0,1,2,3,4
0,-0.095847,2.016646,0.215286,-0.935430,0.245228
1,-0.604492,-0.299550,1.793911,-1.370485,0.546791
2,1.262095,-0.895578,-0.635078,-0.302393,-0.389642
3,0.474629,0.011931,1.760115,-1.129628,-0.460183
4,-0.179844,1.996739,-1.317550,-0.169985,-0.343790
5,0.586624,-0.777309,-0.669965,-0.309959,-0.409040
6,0.371966,1.049417,-1.257589,-0.163679,-0.454892
7,0.311302,-0.258565,-0.563124,-0.238080,-0.278539
8,-0.714154,-0.547797,0.537988,2.220928,-1.252006
9,-0.214842,2.117350,-1.556306,0.123835,-0.456656


### Modelling the Data

In [4]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})
performance


,model,Accuracy,Precision,Recall,F1


- Our primary goal is to identify the correct sound i.e, Nasal or Oral., with a high degree of accuracy, then accuracy is the best scoring metric. Therefore we are considering accuracy as the scoring metric.

## Fitting a Logistic Regression Model

In [5]:
log_reg_model = LogisticRegression()
_ = log_reg_model.fit(X_train, np.ravel(y_train))

In [6]:
model_preds = log_reg_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"default logistic", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

### Logistic Regression using RandomSearchCV 

In [7]:
hyperparam= {'C':[0.001, 0.01, 0.1, 1, 10, 100],
                      'penalty':['l2', 'l1']
                     }

score_measure = "accuracy"
#create a logistic regression model
logistic_model = LogisticRegression()

#create a random search cv object
random_search = RandomizedSearchCV(estimator=logistic_model,
                                     param_distributions=hyperparam,
                                     cv=5,scoring=score_measure,
                                     n_iter=10,
                                     n_jobs=-1,
                                   )

_ = random_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {random_search.best_score_}")
print(f"... with parameters: {random_search.best_params_}")

bestRecallTree = random_search.best_estimator_


The best accuracy score is 0.7506746288258893
... with parameters: {'penalty': 'l2', 'C': 100}


C:\Users\psrik\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\psrik\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\psrik\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\psrik\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
Valu

In [8]:
c_matrix = confusion_matrix(y_test, random_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

performance = pd.concat([performance, pd.DataFrame({'model':"LogReg - RandomCV", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

### Logistic Regression using GridsearchCV

In [9]:
score_measure = "accuracy"

Log_Reg = LogisticRegression()

#Define the parameter grid
param_grid = {
    'penalty':['l1','l2'],
    'C':[0.1,1,10]
}

#Create the grid search object
grid_searchCV = GridSearchCV(estimator=Log_Reg, param_grid=param_grid, cv = 5, scoring= score_measure )


_ = grid_searchCV.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_searchCV.best_score_}")
print(f"... with parameters: {grid_searchCV.best_params_}")

bestRecallTree = grid_searchCV.best_estimator_

C:\Users\psrik\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\psrik\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\psrik\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\psrik\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

The best accuracy score is 0.7506746288258893
... with parameters: {'C': 1, 'penalty': 'l2'}


C:\Users\psrik\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\psrik\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\psrik\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\psrik\anaconda3\lib\site-pac

In [10]:
c_matrix = confusion_matrix(y_test, grid_searchCV.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

performance = pd.concat([performance, pd.DataFrame({'model':"LogReg - GridCV", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

### Decision Tree classification model using defaults 

In [11]:
decitree = DecisionTreeClassifier().fit(X_train, np.ravel(y_train))

In [12]:
c_matrix = confusion_matrix(y_test, decitree.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Decision Tree using Defaults", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

#### Decision Tree using RandomsearchCV

In [13]:
score_measure = "accuracy"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(1,70),  
    'min_samples_leaf': np.arange(1,70),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 200), 
    'max_depth': np.arange(1,50), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best accuracy score is 0.8290885585003231
... with parameters: {'min_samples_split': 2, 'min_samples_leaf': 3, 'min_impurity_decrease': 0.0026, 'max_leaf_nodes': 102, 'max_depth': 12, 'criterion': 'entropy'}


C:\Users\psrik\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 2500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\psrik\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\psrik\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "C:\Users\psrik\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 250, in fit
    raise ValueError(
ValueError: min_samples_split must be an integer greater than 1 or a float in (

In [14]:
c_matrix = confusion_matrix(y_test, rand_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

performance = pd.concat([performance, pd.DataFrame({'model':"Decision Tree- RandCV", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

#### Decision Tree using GridsearchCV

In [15]:
score_measure = "accuracy"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(30,36),  
    'min_samples_leaf': np.arange(6,12),
    'min_impurity_decrease': np.arange(0.0048, 0.0054, 0.0001),
    'max_leaf_nodes': np.arange(162,168), 
    'max_depth': np.arange(15,21), 
    'criterion': ['entropy'],
}

dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 9072 candidates, totalling 45360 fits
The best accuracy score is 0.8068868573070255
... with parameters: {'criterion': 'entropy', 'max_depth': 15, 'max_leaf_nodes': 162, 'min_impurity_decrease': 0.0048, 'min_samples_leaf': 6, 'min_samples_split': 30}


In [16]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]


performance = pd.concat([performance, pd.DataFrame({'model':"Decision Tree- GridCV", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

#### SVM model using linear kernal

In [17]:
from sklearn.svm import SVC
svm_linear_model = SVC(kernel="linear")
_ = svm_linear_model.fit(X_train, np.ravel(y_train))

In [18]:
model_preds = svm_linear_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"SVM Linear", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

#### SVM Linear Model - RandomSearch CV

In [19]:
score_measure = "accuracy"
# Create parameter grid
param_grid = {'C': [0.1, 1, 10, 100],
              'max_iter': [1000, 1500, 2000]}

#create a SVM classifier
classifier = SVC(kernel='linear')

# Create the random search model
rand_search = RandomizedSearchCV(classifier, param_grid, cv=5, n_iter=50, scoring=score_measure, n_jobs=-1, verbose=1)


_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

C:\Users\psrik\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 12 is smaller than n_iter=50. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 12 candidates, totalling 60 fits
The best accuracy score is 0.751584736458686
... with parameters: {'max_iter': 1500, 'C': 1}


C:\Users\psrik\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\psrik\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [20]:
c_matrix = confusion_matrix(y_test, rand_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

performance = pd.concat([performance, pd.DataFrame({'model':"SVM Linear- RandCV", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

#### SVM Linear using GridSearchCV

In [21]:
score_measure = "accuracy"
kfolds = 5
#create a SVM classifier
clf = SVC(kernel='linear')
#define the parameter grid
param_grid = {'C': [0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01, 0.001], 
              'kernel': ['linear']}  
#instantiate the GridSearchCV object
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring= score_measure)


_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_


C:\Users\psrik\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\psrik\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\psrik\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\psrik\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

The best accuracy score is 0.7479566185448537
... with parameters: {'C': 0.1, 'gamma': 1, 'kernel': 'linear'}


C:\Users\psrik\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

performance = pd.concat([performance, pd.DataFrame({'model':"SVM Linear- GridCV", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

#### SVM classification model using rbf kernal

In [23]:
svm_rbf_model = SVC(kernel="rbf", C=10, gamma='scale')
_ = svm_rbf_model.fit(X_train, np.ravel(y_train))

In [24]:
model_preds = svm_rbf_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"SVM rbf", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

#### SVM rbf - RandomSearchCV

In [25]:
score_measure = "accuracy"
param_grid = {'C': [0.1, 1, 10, 100],
              'max_iter': [1000, 1500, 2000]}

#create a SVM classifier
svm_clf = SVC(kernel='rbf')

# Create the random search model
random_search_rbf = RandomizedSearchCV(svm_clf, param_grid, cv=5, n_iter=50, scoring=score_measure, n_jobs=-1, verbose=1)


_ = random_search_rbf.fit(X_train, y_train)

print(f"The best {score_measure} score is {random_search_rbf.best_score_}")
print(f"... with parameters: {random_search_rbf.best_params_}")

bestRecallTree = random_search_rbf.best_estimator_

C:\Users\psrik\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 12 is smaller than n_iter=50. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 12 candidates, totalling 60 fits
The best accuracy score is 0.8381650095935811
... with parameters: {'max_iter': 1500, 'C': 10}


C:\Users\psrik\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\psrik\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [26]:
c_matrix = confusion_matrix(y_test, random_search_rbf.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

performance = pd.concat([performance, pd.DataFrame({'model':"SVM rbf- RandCV", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

### SVM rbf - GridSearchCV

In [27]:
score_measure = "accuracy"
clf = SVC(kernel='rbf')

#define the parameter grid
param_grid = {'C': [0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01, 0.001], 
              'kernel': ['rbf']} 


grid_CV = GridSearchCV(clf, param_grid, cv=5, scoring= score_measure)


_ = grid_CV.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_CV.best_score_}")
print(f"... with parameters: {grid_CV.best_params_}")

bestRecallTree = grid_CV.best_estimator_


C:\Users\psrik\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\psrik\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\psrik\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\psrik\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

The best accuracy score is 0.8612819486769068
... with parameters: {'C': 10, 'gamma': 1, 'kernel': 'rbf'}


In [28]:
c_matrix = confusion_matrix(y_test, grid_CV.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

performance = pd.concat([performance, pd.DataFrame({'model':"SVM rbf- GridCV", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

#### SVM classification model using polynomial kernal

In [29]:
svm_poly_model = SVC(kernel="poly", degree=3, coef0=1, C=10, probability = True)
_ = svm_poly_model.fit(X_train, np.ravel(y_train))

In [30]:
model_preds = svm_poly_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"poly svm", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

### SVM Poly - RandomSearchCV

In [31]:
score_measure = "accuracy"
param_grid = {'C': [0.1, 1, 10, 100],
              'max_iter': [1000, 1500, 2000]}

svm_clf = SVC(kernel='poly')

# Create the random search model
rand_search_polym = RandomizedSearchCV(svm_clf, param_grid, cv=5, n_iter=50, scoring=score_measure, n_jobs=-1, verbose=1)


_ = rand_search_polym.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search_polym.best_score_}")
print(f"... with parameters: {rand_search_polym.best_params_}")

bestRecallTree = rand_search_polym.best_estimator_

C:\Users\psrik\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 12 is smaller than n_iter=50. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 12 candidates, totalling 60 fits
The best accuracy score is 0.7615497481043699
... with parameters: {'max_iter': 2000, 'C': 1}


C:\Users\psrik\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\psrik\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [32]:
c_matrix = confusion_matrix(y_test, rand_search_polym.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

performance = pd.concat([performance, pd.DataFrame({'model':"SVM Poly- RandCV", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

#### SVM Poly - GridSearchCV

In [33]:
score_measure = "accuracy"
clf = SVC(kernel='poly')

#define the parameter grid
param_grid = {'C': [0.1, 1, 10, 100],  
              'gamma': [1, 0.1, 0.01, 0.001], 
              'kernel': ['poly']}  
#instantiate the GridSearchCV object
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring= score_measure)


_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

C:\Users\psrik\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\psrik\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\psrik\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\psrik\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

The best accuracy score is 0.7620083931008301
... with parameters: {'C': 100, 'gamma': 1, 'kernel': 'poly'}


In [34]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]

performance = pd.concat([performance, pd.DataFrame({'model':"SVM Poly- GridCV", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

In [36]:
TP

440

In [37]:
TN

739

In [38]:
FP

400

In [39]:
FN

43

In [35]:
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.717016,0.516000,0.801242,0.627737
0,LogReg - RandomCV,0.717016,0.516000,0.801242,0.627737
0,LogReg - GridCV,0.717016,0.516000,0.801242,0.627737
0,Decision Tree using Defaults,0.844636,0.694118,0.855072,0.766234
0,Decision Tree- RandCV,0.818126,0.663763,0.788820,0.720908
0,Decision Tree- GridCV,0.808878,0.627019,0.884058,0.733677
0,SVM Linear,0.709001,0.506765,0.853002,0.635802
0,SVM Linear- RandCV,0.688656,0.487028,0.855072,0.620586
0,SVM Linear- GridCV,0.709618,0.507371,0.855072,0.636854
0,SVM rbf,0.822441,0.641509,0.915114,0.754266


## Analysis

- The overall accuracy across all of the models is fairly high, ranging from 0.709001 to 0.848952. The Decision Tree using Defaults model and the SVM rbf model have the second and thrid accuracies,at 0.8446 and 0.822. Therefore, it can be concluded that theSVM with kernal ='rbf' using GridSearchCV(i.e. 0.8489) is the best performing model out of the ones tested.